# 월마트 판매

## #01. 준비작업

### [1] 패키지 참조

In [223]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(work_path)

from helper.regrassion import *
from helper.util import *
from helper.plot import *
from helper.analysis import *

### [2] 데이터 가져오기

주요 소매업체의 매출에 영향을 미칠 수 있는 요소 찾기

https://www.kaggle.com/datasets/mikhail1681/walmart-sales


| 변수명 | 설명 |
|:--:|--:|
| Store | Store number |
| Date | Sales week start date |
| Weekly_Sales | Store week sales |
| Holiday_Flag | Mark on the presence or absence of a holiday |
| Temperature | Air temperature in the region |
| Fuel_Price | Fuel cost in the region |
| CPI | Consumer price index(소비자 물가 지수) |
| Unemployment | Unemployment rate |

In [224]:
origin = my_read_data("./res/Walmart_sales.csv", info = False)
my_pretty_table(origin.head())

+----+---------+------------+----------------+----------------+---------------+--------------+---------+----------------+
|    |   Store | Date       |   Weekly_Sales |   Holiday_Flag |   Temperature |   Fuel_Price |     CPI |   Unemployment |
|----+---------+------------+----------------+----------------+---------------+--------------+---------+----------------|
|  0 |       1 | 05-02-2010 |    1.64369e+06 |              0 |         42.31 |        2.572 | 211.096 |          8.106 |
|  1 |       1 | 12-02-2010 |    1.64196e+06 |              1 |         38.51 |        2.548 | 211.242 |          8.106 |
|  2 |       1 | 19-02-2010 |    1.61197e+06 |              0 |         39.93 |        2.514 | 211.289 |          8.106 |
|  3 |       1 | 26-02-2010 |    1.40973e+06 |              0 |         46.63 |        2.561 |  211.32 |          8.106 |
|  4 |       1 | 05-03-2010 |    1.55481e+06 |              0 |          46.5 |        2.625 |  211.35 |          8.106 |
+----+---------+--------

In [225]:
df = my_replace_outliner_to_nan(origin, 'Weekly_Sales').dropna()
df

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date
0,1,1643690.90,0,42.31,2.572,211.096358,8.106,05-02-2010
1,1,1641957.44,1,38.51,2.548,211.242170,8.106,12-02-2010
2,1,1611968.17,0,39.93,2.514,211.289143,8.106,19-02-2010
3,1,1409727.59,0,46.63,2.561,211.319643,8.106,26-02-2010
4,1,1554806.68,0,46.50,2.625,211.350143,8.106,05-03-2010
...,...,...,...,...,...,...,...,...
6430,45,713173.95,0,64.88,3.997,192.013558,8.684,28-09-2012
6431,45,733455.07,0,64.89,3.985,192.170412,8.667,05-10-2012
6432,45,734464.36,0,54.47,4.000,192.327265,8.667,12-10-2012
6433,45,718125.53,0,56.47,3.969,192.330854,8.667,19-10-2012


In [226]:
# 주차별 평균 / 중간값

df_date = df[['Date','Weekly_Sales']].groupby('Date').mean().sort_values('Weekly_Sales').reset_index()
map_date = {}
for i,v in enumerate(df_date['Date']):
    map_date[v] = i//48
df['Date_value'] = df['Date'].map(map_date)
map_date

{'28-01-2011': 0,
 '27-01-2012': 0,
 '31-12-2010': 0,
 '21-01-2011': 0,
 '14-01-2011': 0,
 '24-09-2010': 0,
 '13-01-2012': 0,
 '20-01-2012': 0,
 '30-09-2011': 0,
 '01-10-2010': 0,
 '23-09-2011': 0,
 '07-01-2011': 0,
 '25-03-2011': 0,
 '22-10-2010': 0,
 '17-09-2010': 0,
 '15-10-2010': 0,
 '01-04-2011': 0,
 '29-04-2011': 0,
 '29-10-2010': 0,
 '29-07-2011': 0,
 '30-04-2010': 0,
 '27-04-2012': 0,
 '28-09-2012': 0,
 '16-09-2011': 0,
 '26-02-2010': 0,
 '20-05-2011': 0,
 '27-07-2012': 0,
 '25-02-2011': 0,
 '26-03-2010': 0,
 '14-09-2012': 0,
 '21-09-2012': 0,
 '14-10-2011': 0,
 '11-03-2011': 0,
 '30-07-2010': 0,
 '23-04-2010': 0,
 '18-03-2011': 0,
 '23-07-2010': 0,
 '06-01-2012': 0,
 '15-04-2011': 0,
 '19-03-2010': 0,
 '23-03-2012': 0,
 '20-04-2012': 0,
 '08-10-2010': 0,
 '21-05-2010': 0,
 '19-10-2012': 0,
 '19-11-2010': 0,
 '16-04-2010': 0,
 '30-03-2012': 0,
 '22-07-2011': 1,
 '27-05-2011': 1,
 '14-05-2010': 1,
 '02-09-2011': 1,
 '13-05-2011': 1,
 '15-07-2011': 1,
 '26-10-2012': 1,
 '10-09-20

In [227]:
df_store = df[['Store','Weekly_Sales']].groupby('Store').mean().sort_values('Weekly_Sales').reset_index()
map_store = {}
for i,v in enumerate(df_store['Store']):
    map_store[v] = i//15
df['Store_value'] = df['Store'].map(map_store)
map_store


{33: 0,
 44: 0,
 5: 0,
 36: 0,
 38: 0,
 3: 0,
 30: 0,
 37: 0,
 16: 0,
 29: 0,
 9: 0,
 42: 0,
 7: 0,
 15: 0,
 43: 0,
 25: 1,
 21: 1,
 45: 1,
 17: 1,
 8: 1,
 35: 1,
 40: 1,
 34: 1,
 26: 1,
 12: 1,
 22: 1,
 18: 1,
 32: 1,
 41: 1,
 28: 1,
 11: 2,
 24: 2,
 23: 2,
 31: 2,
 19: 2,
 39: 2,
 1: 2,
 6: 2,
 27: 2,
 10: 2,
 2: 2,
 13: 2,
 14: 2,
 4: 2,
 20: 2}

In [228]:
df2 = df.drop(['Store','Date'],axis=True)
df2

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value
0,1643690.90,0,42.31,2.572,211.096358,8.106,2,2
1,1641957.44,1,38.51,2.548,211.242170,8.106,2,2
2,1611968.17,0,39.93,2.514,211.289143,8.106,2,2
3,1409727.59,0,46.63,2.561,211.319643,8.106,0,2
4,1554806.68,0,46.50,2.625,211.350143,8.106,1,2
...,...,...,...,...,...,...,...,...
6430,713173.95,0,64.88,3.997,192.013558,8.684,0,1
6431,733455.07,0,64.89,3.985,192.170412,8.667,2,1
6432,734464.36,0,54.47,4.000,192.327265,8.667,1,1
6433,718125.53,0,56.47,3.969,192.330854,8.667,0,1


In [229]:
# my_pairplot(df2)

In [230]:
df2['Holiday_Flag'] = df2['Holiday_Flag'].astype('uint8') 
df2.dtypes

Weekly_Sales    float64
Holiday_Flag      uint8
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
Date_value        int64
Store_value       int64
dtype: object

In [231]:
temperature = []
for i in df2['Temperature']:
    temperature.append((i-32)*5/9)
df2['Temperature'] = temperature
df2.columns

Index(['Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Date_value', 'Store_value'],
      dtype='object')

In [232]:
df2['Temperature2'] = df2['Temperature']**2
df2['Fuel_Price2'] = df2['Fuel_Price']**2
df2['CPI2'] = df2['CPI']**2
df2['Unemployment2'] = df2['Unemployment']**2


In [233]:
df3 = my_replace_outliner_to_nan(df2, 'Weekly_Sales').dropna()
df2.describe()

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value,Temperature2,Fuel_Price2,CPI2,Unemployment2
count,6.401000e+03,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000,6401.000000
mean,1.036130e+06,0.068895,15.984468,3.359634,171.642219,8.002298,0.987658,0.994688,360.174624,11.498428,31010.007154,67.565749
std,5.451961e+05,0.253296,10.231704,0.459696,39.359852,1.878705,0.813943,0.815458,314.226152,3.073361,13620.389517,34.833892
min,2.099862e+05,0.000000,-18.922222,2.472000,126.064000,3.879000,0.000000,0.000000,0.000123,6.110784,15892.132096,15.046641
25%,5.517431e+05,0.000000,8.700000,2.933000,131.784000,6.891000,0.000000,0.000000,79.903735,8.602489,17367.022656,47.485881
50%,9.572983e+05,0.000000,17.144444,3.452000,182.658578,7.874000,1.000000,1.000000,293.931975,11.916304,33364.156190,61.999876
75%,1.414565e+06,0.000000,23.888889,3.737000,212.833640,8.622000,2.000000,2.000000,570.679012,13.965169,45298.158273,74.338884
max,2.685352e+06,1.000000,37.855556,4.468000,227.232807,14.313000,2.000000,2.000000,1433.043086,19.963024,51634.748486,204.861969


In [234]:
my_anova(df3[['Holiday_Flag','Weekly_Sales']],hue = 'Holiday_Flag',target = 'Weekly_Sales')

pingouin.anova
+----+--------------+-------------+------+-------------+---------+-----------+-------------+
|    | Source       |          SS |   DF |          MS |       F |     p-unc |         np2 |
|----+--------------+-------------+------+-------------+---------+-----------+-------------|
|  0 | Holiday_Flag | 1.22323e+12 |    1 | 1.22323e+12 | 4.11732 | 0.0424875 | 0.000643018 |
|  1 | Within       |  1.9011e+15 | 6399 | 2.97094e+11 |     nan |       nan |         nan |
+----+--------------+-------------+------+-------------+---------+-----------+-------------+

statsmodels.anova.anova_lm
+-----------------+------+-------------+-------------+---------+-----------+
|                 |   df |      sum_sq |     mean_sq |       F |    PR(>F) |
|-----------------+------+-------------+-------------+---------+-----------|
| C(Holiday_Flag) |    1 | 1.22323e+12 | 1.22323e+12 | 4.11732 | 0.0424875 |
| Residual        | 6399 |  1.9011e+15 | 2.97094e+11 |     nan |       nan |
+-------------

In [235]:
df4 = my_standard_scaler(df3, 'Weekly_Sales')
df4.head()

,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value,Temperature2,Fuel_Price2,CPI2,Unemployment2,Holiday_Flag,Weekly_Sales
0,-1.002520,-1.713516,1.002474,0.055203,1.243847,1.232915,-1.041902,-1.589018,0.995032,-0.053358,0,1643690.90
1,-1.208867,-1.765728,1.006179,0.055203,1.243847,1.232915,-1.104687,-1.629003,0.999554,-0.053358,1,1641957.44
2,-1.131758,-1.839696,1.007372,0.055203,1.243847,1.232915,-1.084545,-1.685008,1.001011,-0.053358,0,1611968.17
3,-0.767937,-1.737446,1.008147,0.055203,-1.213518,1.232915,-0.936067,-1.607391,1.001957,-0.053358,0,1409727.59
4,-0.774996,-1.598213,1.008922,0.055203,0.015164,1.232915,-0.939787,-1.499389,1.002904,-0.053358,0,1554806.68


In [236]:
yname = 'Weekly_Sales'
x_train, x_test, y_train, y_test = my_train_test_split(df4, yname, test_size=0.2)
my_linear_regrassion(x_train, y_train, x_test, y_test, use_plot=False, report=True)

[회귀식]
Weekly_Sales = 37001.885 * Temperature + 353367.199 * Fuel_Price + 770399.321 * CPI + -43675.533 * Unemployment + 63748.716 * Date_value + 492781.877 * Store_value + -42301.365 * Temperature2 + -359367.716 * Fuel_Price2 + -785536.463 * CPI2 + 68122.987 * Unemployment2 + 36101.852 * Holiday_Flag + 1032933.640

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.837239 |              173808 |         4.87835e+10 |           220870 |                         19.0854 |               -2.9134 |
+--------+----------------+---------------------+---------------------+------------------+------------------------

LinearRegression(n_jobs=-1)

In [244]:
ignore = ['CPI2','Fuel_Price2','Unemployment2','Fuel_Price','Temperature','Holiday_Flag','CPI']
x_train2 = x_train.drop(ignore,axis=1)
x_test2 = x_test.drop(ignore,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test, use_plot=False, report=True)

[회귀식]
Weekly_Sales = 29190.475 * Unemployment + 66058.713 * Date_value + 494341.066 * Store_value + -24448.774 * Temperature2 + 1035794.808

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.832609 |              177196 |         5.01711e+10 |           223989 |                         19.2812 |              -3.41147 |
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+

[선형회귀분석 결과보고]
+----+--------------+--------------+--------------------+------------+------------------+--------+------------+---------+
|   

LinearRegression(n_jobs=-1)